In [1]:
import numpy as np
import cv2
# from moviepy.editor import VideoFileClip
from IPython.display import clear_output, Image, display, HTML
import matplotlib.pyplot as plt
import pysindy as ps

### Load U,V

In [11]:
u_all = np.load("C:/Users/laure/Documents/ARA/laurd_ara/video_processing/improved_fixed_low_piv_U.npy")
v_all = np.load("C:/Users/laure/Documents/ARA/laurd_ara/video_processing/improved_fixed_low_piv_V.npy")
all_data = np.stack((u_all,v_all))
all_data = np.where(np.isfinite(all_data),all_data,0.)
all_data.shape

(2, 161, 17, 13)

In [12]:
all_data= all_data.transpose((2,3,1,0))
print(all_data.shape)
# u[:, :, :, 0] = u_sol
# u[:, :, :, 1] = v_sol
u_dot = ps.FiniteDifference(axis=2)._differentiate(all_data[:,:,:,0], t=1)
v_dot = ps.FiniteDifference(axis=2)._differentiate(all_data[:,:,:,1], t=1)

data_dot= np.stack((u_dot,v_dot)).transpose((2,3,1,0))

t=np.arange(all_data.shape[2])
x= np.arange(all_data.shape[1])
y = np.arange(all_data.shape[0])
X, Y = np.meshgrid(x, y)

# Choose 60 % of data for training because data is big...
# can only randomly subsample if you are passing u_dot to model.fit!!!
# train = np.random.choice(len(t), int(len(t) * 0.6), replace=False)
# test = [i for i in np.arange(len(t)) if i not in train]
# u_train = u[:, :, :, :]
# u_test = u[:, :, test, :]
# u_dot_train = u_dot[:, :, :, :]
# u_dot_test = u_dot[:, :, test, :]
# t_train = t#[train]
# t_test = t[test]
spatial_grid = np.asarray([X, Y]).T

(17, 13, 161, 2)


In [13]:
library_functions = [
    lambda x: x,
    lambda x: x*x,
    lambda x: x * x * x,
    lambda x,y: x*y,
    lambda x, y: x * y * y,
    lambda x, y: x * x * y,
]
library_function_names = [
    lambda x: x,
    lambda x: x + x,
    lambda x: x + x + x,
    lambda x,y: x+y,
    lambda x, y: x + y + y,
    lambda x, y: x + x + y,
]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=2,
    spatial_grid=spatial_grid,
    include_bias=True,
    is_uniform=True,
    #periodic=True
)


In [14]:
spatial_grid.shape

(13, 17, 2)

In [15]:
poly_library = ps.PolynomialLibrary(include_bias=False)
fourier_library = ps.FourierLibrary()

# inputs_per_library = np.array([[0,1],[0,1],[0,0]])
tensor_array = [[1, 1, 0]]

In [16]:
generalized_library = ps.GeneralizedLibrary(
    [poly_library, fourier_library, pde_lib],
    tensor_array=tensor_array,
    # inputs_per_library=inputs_per_library,
)


In [22]:
thresh=1e-2
print('STLSQ model: ')
optimizer = ps.STLSQ(threshold=thresh, alpha=1e-3,max_iter=100)

model = ps.SINDy(feature_library=generalized_library, optimizer=optimizer)
model.fit(all_data, x_dot=data_dot,ensemble=True)
model.print()
# u_dot_stlsq = model.predict(u_test)

STLSQ model: 


c:\Users\laure\anaconda3\lib\site-packages\pysindy\pysindy.py:305: UserWarning: Ensembling arguments are deprecated.Use the EnsembleOptimizer class instead.
  warnings.warn(
c:\Users\laure\anaconda3\lib\site-packages\pysindy\optimizers\base.py:289: UserWarning: n_models must be a positive integer.  Explicitly initialized to zero or None, defaulting to 20.
  warnings.warn(


(x0)' = 0.004 x0^2 + -0.010 x0 x1 + 66.994 x1^2 + 866.272 cos(1 x1) + -866.270 1 + 0.004 x0x0 + 66.994 x1x1 + -0.588 x1x1x1 + -0.010 x0x1 + -0.003 x1_2 + 0.002 x1_1 + -0.006 x1_12 + -0.005 x0x1_2 + -0.006 x1x1_2 + -0.006 x0x0x1_2 + -0.015 x0x0x0x1_2 + -0.004 x1x1x1x1_2 + 0.007 x0x1x0_2 + -0.008 x0x1x1x0_2 + -0.011 x0x1x1x1_2 + 0.013 x0x0x1x0_2 + -0.006 x0x0x1x1_2 + -0.002 x0x1_22 + -0.001 x1x1_22 + 0.001 x0x0x1_22 + 0.001 x1x1x0_22 + 0.001 x0x0x0x1_22 + -0.039 x1x1x1x0_22 + -0.049 x0x1x0_22 + -0.020 x0x1x1x0_22 + -0.011 x0x1x1x1_22 + -0.004 x0x0x1x0_22 + -0.001 x0x0x1x1_22 + 0.026 x0x1_1 + -0.010 x0x0x1_1 + 0.017 x1x1x0_1 + 0.007 x1x1x1_1 + 0.002 x0x0x0x0_1 + -0.012 x1x1x1x0_1 + -0.001 x1x1x1x1_1 + -0.009 x0x1x1x0_1 + -0.018 x0x1x1x1_1 + -0.003 x0x0x1x1_1 + 0.023 x0x1_12 + 0.049 x1x1_12 + 0.027 x0x0x1_12 + 0.005 x1x1x0_12 + 0.001 x0x0x0x0_12 + -0.007 x0x0x0x1_12 + 0.013 x1x1x1x0_12 + -0.005 x0x1x0_12 + 0.004 x0x1x1x1_12 + -0.014 x0x0x1x0_12 + 0.006 x0x0x1x1_12 + 0.013 x0x1_11 + 0.017 x